In [12]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
from scipy import stats
import os
import datetime
import statsmodels.formula.api as smf
from dateutil.parser import parse
import statsmodels.api as sm

summary of our work
- 延续前面的代码

reference from chap05_table8_14.ipynb

In [14]:
# 因为回归是用年度数据

# 读取月度因子数据
factor_monthly = pd.read_csv('F-F_Year_Factors.csv')
factor_monthly['dateff'] = factor_monthly['dateff'].apply(int).apply(str).apply(parse)
factor_monthly = factor_monthly[(factor_monthly['dateff'] >= datetime.datetime(1987, 12, 31, 0, 0))&(factor_monthly['dateff'] <= datetime.datetime(2013, 1, 1, 0, 0))]
factor_monthly.rename(columns = {'dateff':'month','mktrf':'mkt'},inplace = True)
factor_monthly = factor_monthly[['month','mkt','smb','hml','umd']]

# 由于常用因子值也为根据变量值分组投资组合的收益率，因此将月度数据按年度连乘(+1后)计算年度因子值
factor_yearly = factor_monthly.copy()
factor_yearly['year'] = factor_yearly['month'].apply(lambda x:x.year)
factor_yearly[['mkt','smb','hml','umd']] = factor_yearly[['mkt','smb','hml','umd']].applymap(lambda x:x+1)
factor_yearly = factor_yearly.groupby('year').prod()
factor_yearly = factor_yearly.applymap(lambda x:(x-1)*100)
factor_yearly = factor_yearly.reset_index()

# 由于个股数据表中中汇报的收益率为t+1期，故此处年份减1使因子值也为t+1期，便于后面连接
factor_yearly_yearlag1 = factor_yearly.copy()
factor_yearly_yearlag1['year'] = factor_yearly_yearlag1['year']-1

In [16]:
# 读取个股数据
all_data = pd.read_csv('alldata2021.csv',index_col=0)
all_data = all_data.reset_index()
all_data = all_data.drop_duplicates(subset=['PERMNO','year'])
all_data = all_data[['PERMNO', 'year', 'beta', 'rt_1', 'BM','MktCap']]
all_data['rt_1'] = all_data['rt_1']*100

# 分别提取不同变量所需分组数据至多个dataframe
proxy_name_list = ['beta','MktCap','BM']
def select_proxy(proxy_name_list):
    for proxy_name in proxy_name_list:
        globals()[proxy_name] = all_data[['PERMNO', 'year','rt_1',proxy_name]]
select_proxy(proxy_name_list)

## 做等权重的7分组
# 根据bm,beta,mktcap分别进行单变量7分组
def mutate_group(proxy_name_list):
    for proxy_name in proxy_name_list:
        globals()[proxy_name] = globals()[proxy_name].dropna()

        quantiles_proxy = globals()[proxy_name].groupby('year')[proxy_name].describe(
        percentiles=[0.1,0.2,0.4,0.6,0.8,0.9]).reset_index()[['year','10%','20%','40%','60%','80%','90%']]

        df= pd.merge(globals()[proxy_name], quantiles_proxy, how = 'left', on = 'year')

        globals()[proxy_name]['group'] = np.select([df[proxy_name] <= df['10%'],
                           (df[proxy_name] > df['10%']) & (df[proxy_name] <= df['20%']),
                           (df[proxy_name] > df['20%']) & (df[proxy_name] <= df['40%']),
                            (df[proxy_name] > df['40%']) & (df[proxy_name] <= df['60%']),
                            (df[proxy_name] > df['60%']) & (df[proxy_name] <= df['80%']),
                            (df[proxy_name] > df['80%']) & (df[proxy_name] <= df['90%']),
                           (df[proxy_name] > df['90%'])],
                           ['1','2','3','4','5','6','7'])
mutate_group(proxy_name_list)

# 计算等权投资组合收益率
def portfolio_ret(proxy_name_list):
    for proxy_name in proxy_name_list:
        globals()[proxy_name] = globals()[proxy_name].dropna()

        globals()[proxy_name] = globals()[proxy_name].groupby(['group', 'year']).apply(
            lambda x: np.average(x['rt_1'], weights=None)).reset_index()
        globals()[proxy_name].rename(columns={0: 'ret_excess'}, inplace=True)
        globals()[proxy_name] = pd.pivot(globals()[proxy_name], index='year', columns='group')[
            'ret_excess'].reset_index()

        # 没有股票的组用0填充,此算例中数据分布基本均匀且无明显缺失，应用不到
        globals()[proxy_name].replace(np.nan,0,inplace = True)

        globals()[proxy_name]['7-1'] = globals()[proxy_name]['7'] - globals()[proxy_name]['1']
portfolio_ret(proxy_name_list)

# 定义NW调整函数
def nw_adjust(df, group, lags=6):
    df.dropna(subset = [group], inplace = True)
    adj_a = np.array(df[group])
    # 对常数回归
    model = sm.OLS(adj_a, [1] * len(adj_a)).fit(cov_type='HAC', cov_kwds={'maxlags': lags})
    return adj_a.mean().round(2), format(float(model.tvalues), ".2f"),float(model.bse) #第三个参数为系数标准差，此处用不上

In [ ]:
# beta

In [17]:
# generate table8
table8_index = ['Beta','','MKtCap','','BM','']
table8_columns = ['1','2','3','4','5','6','7','7-1']
table8 = pd.DataFrame(index = table8_index, columns = table8_columns)
table8.columns.name = 'Sort Variable'
def generate_table8(proxy_name_list):
    for i in range(len(proxy_name_list)):
        for j in range(len(table8_columns)):
            table8.iloc[2*i,j] = nw_adjust(globals()[proxy_name_list[i]], table8_columns[j])[0]
            table8.iloc[2*i+1,j] = nw_adjust(globals()[proxy_name_list[i]],table8_columns[j])[1]
generate_table8(proxy_name_list)
table8

Sort Variable,1,2,3,4,5,6,7,7-1
Beta,16.73,14.09,14.82,12.91,12.22,11.0,10.52,-6.21
,4.62,5.09,5.17,5.65,5.36,4.66,3.02,-1.44
MKtCap,29.11,16.54,12.33,10.41,8.69,9.28,8.33,-20.78
,6.58,4.68,3.93,4.42,4.86,5.14,4.02,-5.07
BM,5.48,7.16,9.84,11.21,15.02,19.03,25.87,20.39
,1.87,3.03,4.80,5.30,6.68,6.68,6.87,8.37


In [18]:
# generate table9
table9_index = ['Excess return','',
                'CAPM','','','',
                'FF','','','','','','','',
                'FFC','','','','','','','','','']
table9_columns = ['Coefficient','1','2','3','4','5','6','7','7-1']
table9 = pd.DataFrame(index = table9_index, columns = table9_columns)
table9.columns.name = 'Model'
table9['Coefficient'] = ['Excess return','',
                         'alpha','','MKT','',
                         'alpha','','MKT','','SMB','','HML','',
                         'alpha','','MKT','','SMB','','HML','','MOM','']

# 连接因子模型因子数据
# 起初在根据年份连接时未如前所述在因子表中将年份减一，导致实际用t期因子值解释t+1期收益，无解释力，
# 因子模型解释后alpha反变大，现已修正
def mutate_factor(proxy_name_list):
    for proxy_name in proxy_name_list:
        globals()[proxy_name] = pd.merge(globals()[proxy_name], factor_yearly_yearlag1, how='left', on=['year'])
mutate_factor(proxy_name_list)

# 定义用于风险调整的CAPM,FF3,FFC因子模型函数
def capm_adjust(df, group, lags = 6):
    df.dropna(subset = [group,'mkt'], inplace = True)
    x = sm.add_constant(df['mkt'])
    y = df[group]
    model = sm.OLS(y, x).fit(cov_type='HAC', cov_kwds={'maxlags': lags})
    return model

def ff3_adjust(df, group, lags = 6):
    df.dropna(subset=[group, 'mkt', 'smb', 'hml'], inplace = True)
    x = sm.add_constant(df[['mkt', 'smb', 'hml']])
    y = df[group]
    model = sm.OLS(y, x).fit(cov_type='HAC', cov_kwds={'maxlags': lags})
    return model

def ffc_adjust(df, group, lags = 6):
    df.dropna(subset=[group, 'mkt', 'smb', 'hml', 'umd'], inplace = True)
    x = sm.add_constant(df[['mkt', 'smb', 'hml','umd']])
    y = df[group]
    model = sm.OLS(y, x).fit(cov_type='HAC', cov_kwds={'maxlags': lags})
    return model

# 分别计算不同因子模型调整后的风险收益及t值
def generate_table9():
    for j in range(len(table9.columns) - 1):
        table9.iloc[0,j+1] = nw_adjust(globals()['beta'], table9_columns[j+1])[0]
        table9.iloc[1,j+1] = nw_adjust(globals()['beta'], table9_columns[j+1])[1]

        for i in range(2):
            table9.iloc[2+2*i, j+1] = capm_adjust(globals()['beta'], table9_columns[j+1]).params[i].round(2)
            table9.iloc[3+2*i, j+1] = capm_adjust(globals()['beta'], table9_columns[j+1]).tvalues[i].round(2)

        for i in range(4):
            table9.iloc[6+2*i, j+1] = ff3_adjust(globals()['beta'], table9_columns[j+1]).params[i].round(2)
            table9.iloc[7+2*i, j+1] = ff3_adjust(globals()['beta'], table9_columns[j+1]).tvalues[i].round(2)

        for i in range(5):
            table9.iloc[14+2*i, j+1] = ffc_adjust(globals()['beta'], table9_columns[j+1]).params[i].round(2)
            table9.iloc[15+2*i, j+1] = ffc_adjust(globals()['beta'], table9_columns[j+1]).tvalues[i].round(2)
generate_table9()
table9


Model,Coefficient,1,2,3,4,5,6,7,7-1
Excess return,Excess return,16.73,14.09,14.82,12.91,12.22,11.0,10.52,-6.21
,,4.62,5.09,5.17,5.65,5.36,4.66,3.02,-1.44
CAPM,alpha,9.12,6.95,6.99,5.45,3.83,0.5,-2.37,-11.5
,,1.93,1.56,1.61,1.5,1.27,0.22,-1.09,-1.96
,MKT,1.03,0.96,1.06,1.01,1.13,1.42,1.74,0.71
,,3.63,3.45,4.26,4.28,5.7,8.18,8.74,1.99
FF,alpha,4.09,2.12,2.78,1.85,1.25,-0.65,-1.25,-5.34
,,1.76,1.59,2.63,2.13,1.05,-0.41,-0.53,-1.7
,MKT,1.11,1.07,1.12,1.06,1.13,1.34,1.56,0.44
,,9.05,6.81,9.44,10.61,14.98,13.9,11.35,2.09


In [48]:
# generate table10
# bivariate independent-sort portfolio，
# The first sort variable is beta and the second sort variable is MktCap；
def generate_table10():
    quantiles_proxy_beta = all_data.groupby('year')['beta'].describe(
            percentiles=[0.3, 0.7]).reset_index()[['year', '30%', '70%']]

    quantiles_proxy_mktcap = all_data.groupby('year')['MktCap'].describe(
            percentiles=[0.25, 0.5, 0.75]).reset_index()[['year', '25%', '50%', '75%']]

    beta_mktcap_group = pd.merge(quantiles_proxy_beta, quantiles_proxy_mktcap, on=['year'])
    beta_mktcap_group.rename(columns = {'30%':'B1_1t','70%':'B1_2t',
                                        '25%':'B2_1t','50%':'B2_2t','75%':'B2_3t'},inplace=True)
    return beta_mktcap_group
table10 = generate_table10()
table10.iloc[:,1:] = table10.iloc[:,1:].applymap(lambda x:round(x,2))
table10

,year,B1_1t,B1_2t,B2_1t,B2_2t,B2_3t
0,1988,0.18,0.66,9.24,32.86,147.05
1,1989,0.17,0.69,9.55,35.24,172.07
2,1990,0.23,0.86,6.34,24.47,137.00
3,1991,0.24,0.85,10.14,41.99,212.18
4,1992,0.25,0.97,16.38,61.59,268.27
5,1993,0.28,0.92,22.25,73.19,299.14
6,1994,0.36,0.97,20.14,67.32,272.35
7,1995,0.27,0.90,25.75,89.00,355.28
8,1996,0.32,0.90,28.50,97.32,403.62
9,1997,0.25,0.72,32.89,115.42,480.37


In [53]:
# generate table11
all_data_and_breakpoints = pd.merge(all_data,table10,on='year')
def portfolio_ind_3x4():
    df = all_data_and_breakpoints.copy()
    df = df.dropna(subset=['beta','MktCap'])
    df['X1_group'] = np.select([
        (df['beta'] <= df['B1_1t']),
        (df['beta'] > df['B1_1t']) & (df['beta'] <= df['B1_2t']),
        (df['beta'] > df['B1_2t'])],
        ['1','2','3'])
    df['X2_group'] = np.select([
        (df['MktCap'] <= df['B2_1t']),
        (df['MktCap'] > df['B2_1t']) & (df['MktCap'] <= df['B2_2t']),
        (df['MktCap'] > df['B2_2t']) & (df['MktCap'] <= df['B2_3t']),
        (df['MktCap'] > df['B2_3t'])],
        ['1','2','3','4'])
    return df[['PERMNO','year','beta','rt_1','MktCap','X1_group','X2_group']]
all_data_and_groups = portfolio_ind_3x4()

def generate_table11(group_table):
    n = group_table.groupby(['year', 'X1_group', 'X2_group'])['PERMNO'].count().reset_index().rename(
            columns={'PERMNO': 'n_firms'})
    df = pd.pivot_table(n, index=['year', 'X2_group'], columns='X1_group')['n_firms'].reset_index()
    df.rename(columns={'1':'beta1','2':'beta2','3':'beta3'},inplace=True)
    return df
table11 = generate_table11(all_data_and_groups)
table11

X1_group,year,X2_group,beta1,beta2,beta3
0,1988,1,731,466,216
1,1988,2,541,577,287
2,1988,3,344,696,407
3,1988,4,106,572,814
4,1989,1,726,423,231
...,...,...,...,...,...
95,2011,4,177,540,223
96,2012,1,663,150,72
97,2012,2,138,390,357
98,2012,3,79,447,361


In [94]:
# generate table13
def avg_r(group_table):
    ewret = group_table.groupby(['year','X2_group','X1_group'])['rt_1'].mean().reset_index()
    df = pd.pivot_table(ewret,index=['year','X2_group'],columns='X1_group')['rt_1'].reset_index()
    df['diff'] = df['3']-df['1']
    df['avg'] = (df['1']+df['2']+df['3'])/3
    df.loc[:,'1':'avg']= df.loc[:,'1':'avg'].applymap(lambda x:round(x,2))
    return df
table13 = avg_r(all_data_and_groups)

def get_diff(table):
    X = pd.DataFrame()
    for i in range(1988,2012):
        x = table[table['year']==i]
        x1 = pd.DataFrame(columns=x.columns,index=list(range(2)))
        x1['year'] = [i,i]
        x1.iloc[0,1] = ['mktcap_diff']
        x1.iloc[1,1] = ['mktcap_avg']
        x1.iloc[0, 2:] = (x.iloc[3, 2:] - x.iloc[0, 2:]).apply(lambda x:round(x,2))
        x1.iloc[1, 2:] = x.iloc[0:3, 2:].mean().apply(lambda x:round(x,2))
        X = pd.concat([X, x])
        X = pd.concat([X, x1])
    return X
table13 = get_diff(table13)
table13 = table13.reset_index(drop= True)
table13

X1_group,year,X2_group,1,2,3,diff,avg
0,1988,1,1.25,-2.22,2.03,0.78,0.35
1,1988,2,-1.0,0.32,-11.29,-10.28,-3.99
2,1988,3,3.9,3.77,4.65,0.75,4.11
3,1988,4,9.62,14.3,17.24,7.63,13.72
4,1988,mktcap_diff,8.37,16.52,15.21,6.85,13.37
...,...,...,...,...,...,...,...
139,2011,2,27.45,21.25,19.17,-8.28,22.62
140,2011,3,10.99,16.87,19.92,8.93,15.92
141,2011,4,12.04,17.68,17.53,5.49,15.75
142,2011,mktcap_diff,-16.04,6.42,27.88,43.92,6.09


In [181]:
# generate table14
##计算综合excess return和FFC调整alpha
table14_sub_index = ['Excess-return','','FFC_alpha','']
table14_sub_column = ['beta1','beta2','beta3','beta Diff','beta Avg']
table14_sub = pd.DataFrame(index=table14_sub_index,columns=table14_sub_column)
subs = table13['X2_group'].unique()

def generate_table14_subs():
    for sub in subs:
        globals()[sub + '_table'] = table14_sub.copy()
        globals()[sub + '_data'] = table13[table13['X2_group']==sub].copy()
        globals()[sub + '_data'].loc[:,'1':'avg'] = globals()[sub + '_data'].loc[:,'1':'avg'].applymap(lambda x:np.float64(x))
        globals()[sub + '_data'] = pd.merge(globals()[sub + '_data'],factor_yearly_yearlag1,on='year')
        for i in range(len(globals()[sub + '_table'].columns)):
            globals()[sub + '_table'].iloc[0,i] = nw_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i])[0]
            globals()[sub + '_table'].iloc[1,i] = nw_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i])[1]
            globals()[sub + '_table'].iloc[2,i] = ffc_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i]).params[0].round(2)
            globals()[sub + '_table'].iloc[3,i] = ffc_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i]).tvalues[0].round(2)
generate_table14_subs()

table14 = pd.DataFrame()
for sub in subs:
    table14 = pd.concat([table14, globals()[sub + '_table']])
table14 = table14.reset_index().reset_index()
table14['level_0'] = ['MktCap 1','','','',
                'MktCap 2', '', '', '',
                'MktCap 3', '', '', '',
                'MktCap 4', '', '', '',
                'MktCap Diff', '', '', '',
                'MktCap Avg', '', '', '',]
table14.rename(columns={'index':'Coefficient',
                        'level_0':''},inplace=True)
table14

,,Coefficient,beta1,beta2,beta3,beta Diff,beta Avg
0,MktCap 1,Excess-return,21.03,24.01,20.67,-0.36,21.9
1,,,5.66,5.58,3.28,-0.09,5.07
2,,FFC_alpha,12.73,19.05,8.6,-4.13,13.46
3,,,5.02,6.38,1.22,-0.76,3.58
4,MktCap 2,Excess-return,11.39,14.0,11.92,0.53,12.43
5,,,3.91,3.79,3.78,0.16,4.34
6,,FFC_alpha,0.72,2.38,1.56,0.85,1.55
7,,,0.37,0.82,0.65,0.22,1.14
8,MktCap 3,Excess-return,9.0,10.66,8.67,-0.33,9.44
9,,,2.96,4.77,3.90,-0.09,4.59


 ## 5.2.6 Presenting the Results

### Single Analysis

One common approach is to present only the average values (Y) for each of the n P1 × n P2 portfolios, and to present both values of Y and the associated t-statistics for the difference and average portfolios.

In [205]:
def generate_table15():
    table15=table14.loc[[2,3,6,7,10,11,14,15,18,19,22,23]]
    table15.index = ['MktCap 1','',
                'MktCap 2', '', 
                'MktCap 3', '', 
                'MktCap 4', '',
                'MktCap Diff', '',
                'MktCap Avg', '']
    table15=table15.drop('Coefficient',axis=1)
    return table15
table15=generate_table15()
table15

,,beta1,beta2,beta3,beta Diff,beta Avg
MktCap 1,,12.85,18.6,9.01,-3.84,13.49
,,5.04,6.37,1.29,-0.71,3.59
MktCap 2,,0.76,2.25,1.61,0.86,1.54
,,0.39,0.75,0.68,0.22,1.13
MktCap 3,,-0.29,1.48,-1.42,-1.12,-0.08
,,-0.1,1.62,-1.18,-0.28,-0.09
MktCap 4,,-0.42,2.26,-1.16,-0.74,0.23
,,-0.32,3.17,-0.69,-0.33,0.29
MktCap Diff,,-13.27,-16.33,-10.18,3.09,-13.26
,,-3.78,-5.5,-1.53,0.86,-3.24


**TABLE 5.15 Bivariate Independent-Sort Portfolio Results**
This table presents the average abnormal returns relative to the FFC model for portfolios sorted independently into three 𝛽 groups and four MktCap. The breakpoints for the 𝛽 portfolios are the 30th and 70th percentiles. The breakpoints for the MktCap portfolios are the 25th, 50th, and 75th percentiles. Table values indicate the alpha relative to the FFC model with corresponding t-statistics in parentheses.

### multiple Analyses, Same Relation of Interes

In [62]:
# FFC_beakpoints
# Fun: 计算 breakpoints
# 1 beta 2 MktCap 3BM
def FFC_beakpoints():
    quantiles_proxy_beta = all_data.groupby('year')['beta'].describe(
            percentiles=[0.3, 0.7]).reset_index()[['year', '30%', '70%']]
    quantiles_proxy_bm = all_data.groupby('year')['BM'].describe(
            percentiles=[0.3, 0.7]).reset_index()[['year', '30%', '70%']]
    quantiles_proxy_mktcap = all_data.groupby('year')['MktCap'].describe(
            percentiles=[0.25, 0.5, 0.75]).reset_index()[['year', '25%', '50%', '75%']]

    FFC_beakpoints = pd.merge(quantiles_proxy_beta, quantiles_proxy_mktcap, on=['year'])
    FFC_beakpoints = pd.merge(FFC_beakpoints, quantiles_proxy_bm, on=['year'])
    FFC_beakpoints.rename(columns = {'30%_x':'B1_1t','70%_x':'B1_2t',
                                        '25%':'B2_1t','50%':'B2_2t','75%':'B2_3t','30%_y':'B3_1t','70%_y':'B3_2t',},inplace=True)
    return FFC_beakpoints
FFC_beakpoints = FFC_beakpoints()
FFC_beakpoints

,year,B1_1t,B1_2t,B2_1t,B2_2t,B2_3t,B3_1t,B3_2t
0,1988,0.179025,0.658815,9.238531,32.856250,147.050625,0.377767,0.898919
1,1989,0.172073,0.694472,9.555000,35.238500,172.070125,0.338303,0.864693
2,1990,0.229329,0.857409,6.336313,24.470375,136.997937,0.368806,1.001725
3,1991,0.237511,0.848384,10.136302,41.994250,212.184000,0.310514,0.922570
4,1992,0.253065,0.969653,16.381000,61.589750,268.274875,0.305796,0.813493
5,1993,0.281613,0.919260,22.246625,73.191250,299.145000,0.284409,0.710897
6,1994,0.357824,0.971374,20.140844,67.319000,272.347188,0.315586,0.734753
7,1995,0.269051,0.899893,25.746500,88.998000,355.283250,0.295128,0.733674
8,1996,0.320785,0.904879,28.495516,97.315250,403.623750,0.267743,0.675970
9,1997,0.254654,0.718753,32.885625,115.416250,480.372750,0.265969,0.643730


In [79]:
# FFC_classify 
#func：所有股票独立分组标签

all_data_and_breakpoints = pd.merge(all_data,FFC_beakpoints,on='year')
def portfolio_ind_FFC():
    df = all_data_and_breakpoints.copy()
    df = df.dropna(subset=['beta','MktCap','BM'])
    df['X1_group'] = np.select([
        (df['beta'] <= df['B1_1t']),
        (df['beta'] > df['B1_1t']) & (df['beta'] <= df['B1_2t']),
        (df['beta'] > df['B1_2t'])],
        ['1','2','3'])
    df['X2_group'] = np.select([
        (df['MktCap'] <= df['B2_1t']),
        (df['MktCap'] > df['B2_1t']) & (df['MktCap'] <= df['B2_2t']),
        (df['MktCap'] > df['B2_2t']) & (df['MktCap'] <= df['B2_3t']),
        (df['MktCap'] > df['B2_3t'])],
        ['1','2','3','4'])
    df['X3_group'] = np.select([
        (df['beta'] <= df['B3_1t']),
        (df['beta'] > df['B3_1t']) & (df['beta'] <= df['B3_2t']),
        (df['beta'] > df['B3_2t'])],
        ['1','2','3'])
    return df[['PERMNO','year','beta','rt_1','MktCap','X1_group','X2_group','X3_group']]
all_data_and_FFC = portfolio_ind_FFC()

,PERMNO,year,beta,rt_1,MktCap,X1_group,X2_group,X3_group
0,10001,1988,0.255128,60.422576,6.362250,2,1,1
3,10005,1988,0.480927,-41.703377,0.785250,2,1,2
8,10010,1988,0.362772,11.074425,15.140250,2,2,1
10,10012,1988,0.007413,-29.582040,4.765406,1,1,1
11,10016,1988,0.520995,-19.131575,400.414000,2,4,2
...,...,...,...,...,...,...,...,...
223349,93428,2012,1.631357,NaN,1014.079290,3,3,3
223350,93429,2012,0.478822,NaN,2571.033120,1,4,2
223353,93433,2012,1.804476,NaN,19.160530,3,1,3
223354,93434,2012,0.486253,NaN,60.779560,1,1,2


In [66]:
''''''
def Mkt_BM(group_table):
    n = group_table.groupby(['year', 'X3_group', 'X2_group'])['PERMNO'].count().reset_index().rename(
            columns={'PERMNO': 'n_firms'})
    df = pd.pivot_table(n, index=['year', 'X2_group'], columns='X3_group')['n_firms'].reset_index()
    df.rename(columns={'1':'BM1','2':'BM2','3':'BM3'},inplace=True)
    
    return df
Mkt_BM(all_data_and_FFC)
''''''

''

In [81]:
## 类似table13
def result_Mkt_BM():
    ewret = all_data_and_FFC.groupby(['year','X2_group','X3_group'])['rt_1'].mean().reset_index()
    df = pd.pivot_table(ewret,index=['year','X2_group'],columns='X3_group')['rt_1'].reset_index()
    df['diff'] = df['3']-df['1']
    df['avg'] = (df['1']+df['2']+df['3'])/3
    df.loc[:,'1':'avg']= df.loc[:,'1':'avg'].applymap(lambda x:round(x,2))
    X = pd.DataFrame()
    for i in range(1988,2012):
        x = df[df['year']==i]
        x1 = pd.DataFrame(columns=x.columns,index=list(range(2)))
        x1['year'] = [i,i]
        x1.iloc[0,1] = ['mktcap_diff']
        x1.iloc[1,1] = ['mktcap_avg']
        x1.iloc[0, 2:] = (x.iloc[3, 2:] - x.iloc[0, 2:]).apply(lambda x:round(x,2))
        x1.iloc[1, 2:] = x.iloc[0:3, 2:].mean().apply(lambda x:round(x,2))
        X = pd.concat([X, x])
        X = pd.concat([X, x1])
    X = get_diff(X)
    X= X.reset_index(drop= True)

    return X

result_Mkt_BM = result_Mkt_BM()
result_Mkt_BM

X3_group,year,X2_group,1,2,3,diff,avg
0,1988,1,-0.25,-2.03,7.25,7.5,1.66
1,1988,2,0.21,-4.92,-11.68,-11.9,-5.46
2,1988,3,6.37,6.64,7.07,0.7,6.69
3,1988,4,16.71,16.88,16.81,0.1,16.8
4,1988,mktcap_diff,16.96,18.91,9.56,-7.4,15.14
...,...,...,...,...,...,...,...
187,2011,4,-3.5,11.2,17.48,20.97,8.39
188,2011,mktcap_diff,-25.41,-12.11,12.78,38.18,-8.25
189,2011,mktcap_avg,27.56,17.24,14.06,-13.5,19.62
190,2011,mktcap_diff,-25.41,-12.11,12.78,38.18,-8.25


In [210]:

table14_sub_index = ['Excess return','','FFC alpha','']
table14_sub_column = ['BM1','BM2','BM3','BM Diff','BM Avg']
table14_sub = pd.DataFrame(index=table14_sub_index,columns=table14_sub_column)
subs = result_Mkt_BM['X2_group'].unique()

def generate_table14_subs():
    for sub in subs:
        globals()[sub + '_table'] = table14_sub.copy()
        globals()[sub + '_data'] = result_Mkt_BM[result_Mkt_BM['X2_group']==sub].copy()
        globals()[sub + '_data'].loc[:,'1':'avg'] = globals()[sub + '_data'].loc[:,'1':'avg'].applymap(lambda x:np.float64(x))
        globals()[sub + '_data'] = pd.merge(globals()[sub + '_data'],factor_yearly_yearlag1,on='year')
        for i in range(len(globals()[sub + '_table'].columns)):
            globals()[sub + '_table'].iloc[0,i] = nw_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i])[0]
            globals()[sub + '_table'].iloc[1,i] = nw_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i])[1]
            globals()[sub + '_table'].iloc[2,i] = ffc_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i]).params[0].round(2)
            globals()[sub + '_table'].iloc[3,i] = ffc_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i]).tvalues[0].round(2)
generate_table14_subs()

table16_1 = pd.DataFrame()
for sub in subs:
    table16_1 = pd.concat([table16_1, globals()[sub + '_table']])
table16_1 = table16_1.reset_index().reset_index()
table16_1['level_0'] = ['MktCap 1','','','',
                'MktCap 2', '', '', '',
                'MktCap 3', '', '', '',
                'MktCap 4', '', '', '',
                'MktCap Diff', '', '', '',
                'MktCap Avg', '', '', '',]
table16_1.rename(columns={'index':'Coefficient',
                        'level_0':''},inplace=True)
table16_1

,,Coefficient,BM1,BM2,BM3,BM Diff,BM Avg
0,MktCap 1,Excess return,23.57,23.17,27.92,4.34,24.88
1,,,6.29,5.51,5.09,1.19,5.94
2,,FFC alpha,18.83,16.84,22.77,3.94,19.48
3,,,8.43,4.68,4.1,0.85,5.92
4,MktCap 2,Excess return,10.94,13.65,12.14,1.2,12.24
5,,,4.17,3.86,3.75,0.43,4.36
6,,FFC alpha,2.02,5.72,0.21,-1.81,2.65
7,,,1.08,2.78,0.1,-0.55,2.18
8,MktCap 3,Excess return,9.02,9.71,8.12,-0.9,8.95
9,,,4.45,4.10,3.34,-0.34,5.14


In [207]:
def generate_table16(table):
    table16=table.loc[[16,17,18,19]]
    return table16
table16_3=generate_table16(table16_1)
table16_2=generate_table16(table14)
table16_3.rename(columns={'BM1':'1','BM2':'2','BM3':'3','BM Diff':'Diff','BM Avg':'Avg'},inplace=True)
table16_3.rename(index={'MktCap Avg':'BM'},inplace=True)
table16_2.rename(columns={'beta1':'1','beta2':'2','beta3':'3','beta Diff':'Diff','beta Avg':'Avg'},inplace=True)
table16=pd.concat([table16_2,table16_3])
table16['type'] = ['beta','','','','BM','','','']
table16

,,Coefficient,1,2,3,Diff,Avg,type
16,MktCap Diff,Excess-return,-14.04,-14.84,-11.89,2.15,-13.59,beta
17,,,-4.65,-3.76,-2.13,0.70,-3.54,
18,,FFC_alpha,-13.09,-16.8,-9.75,3.35,-13.21,
19,,,-3.74,-5.57,-1.45,0.92,-3.22,
16,MktCap Diff,Excess return,-17.64,-14.97,-19.53,-1.89,-17.38,BM
17,,,-3.69,-2.85,-2.79,-0.66,-3.12,
18,,FFC alpha,-19.08,-15.65,-22.79,-3.71,-19.18,
19,,,-4.56,-3.5,-3.34,-1.18,-3.85,


**TABLE 5.16 Bivariate Independent-Sort Portfolio Results—Differences**

Another common approach is to present only the results for the difference or average portfolios. This is frequently the case when the objective of the analysis is to examine the relation between one of the sort variables (say X2) and Y while controlling for the other sort variable (X1), but we are not interested in the relation between X1 and Y when controlling for X2. This presentation style also allows the researcher to present results for more than one bivariate portfolio analysis while minimizing the amount of space required to do so. In exemplifying each of these approaches to presenting the results of bivariate portfolio analyses, we present not only the results for our analysis using 훽 and MktCap as the sort variables, but also the results of a similar analysis using BM and MktCap as the sort variables. In this second analysis, we once again use the 30th and 70th percentiles of BM to calculate the BM breakpoints and the 25th, 50th, and 75th percentiles of MktCap for the MktCap breakpoint

In [243]:
table17_1=table16_1.loc[:,['Coefficient','BM Avg']]
table17_2=table14.loc[:,'beta Avg']
table17=pd.concat([table17_1,table17_2],axis=1)
table17

,Coefficient,BM Avg,beta Avg
0,Excess return,24.88,21.9
1,,5.94,5.07
2,FFC alpha,19.48,13.46
3,,5.92,3.58
4,Excess return,12.24,12.43
5,,4.36,4.34
6,FFC alpha,2.65,1.55
7,,2.18,1.14
8,Excess return,8.95,9.44
9,,5.14,4.59


**TABLE 5.17 Bivariate Independent-Sort Portfolio Results—Averages**


**TABLE 5.18 Bivariate Independent-Sort Portfolio Results—Differences**

**TABLE 5.19 Bivariate Independent-Sort Portfolio Results—Averages**


## 5.3 BIVARIATE DEPENDENT-SORT ANALYSI

### 5.3.1 Breakpoints

The dependent-sort portfolio procedure begins by calculating breakpoints for the first sort variable (X1, the control variable). Letting $n_{P1}$ be the number of groups based on $X1$, and ${p1}_j, j ∈ {1, … , nP1− 1}$be the percentiles used to calculate thebreakpoints, the breakpoints for X1 are calculated exactly as described in Section 5.2 and equation (5.10). As always, breakpoints may be calculated using a different set of entities than the set that will eventually be sorted into portfolios.
    
    
Having calculated the breakpoints for the first sort variable X1 for each time period t, the entities are divided into nP1groups based on the breakpoints B1j,t. The next step is what differentiates dependent-sort portfolio analysis from independent-sort portfolio analysis. In dependent-sort analysis, the second sort, based on values of $X2$, is done separately for each of the nP1groups of entities created by the breakpoints for the first sort variable. Thus, the breakpoints that determine how the sample is divided into portfolios based on the second independent variable X2 will be different for each of the nP1groups formed by sorting on the first sort variable. We therefore define the breakpoints for the second sort variable as


$$B 2_{j, k, t}=\operatorname{Pctl}_{p 2_{k}}\left(\left\{X 2_{t} \mid B 1_{j-1, t} \leq X 1_{t} \leq B 1_{j, t}\right\}\right)$$


where $j ∈ {1, … , nP1}$,$ k ∈ {1, … , nP2− 1}$, ${p2}_k$is the percentile for the kth breakpoint based on the second sort variable, nP2is the number of groups to be formed based on the second sort variable $X2$, ${B1}_{0,t}= −∞, {B1}_{nP1,t}= ∞,$ and $  {X2t|{B1}_{j−1,t} \ ≤X1t≤ \ {B1}_{j,t}} $is the set of values of X2 across all entities in the sample with values of X1 that are between B1j−1,tand B1j,tinclusive. Thus, for each of the nP1groups of entities formed on X1, there will be nP2− 1 breakpoints for the second sort variable X2.

In [182]:


# generate table20
# bivariate independent-sort portfolio
# deirctly sort based on all_data_and_groups

# generate table11
all_data_and_breakpoints = pd.merge(all_data,table10,on='year')


def generate_table20_2():
    df=all_data_and_groups.copy()
    quantiles_proxy_beta = all_data.groupby('year')['beta'].describe(percentiles=[0.3, 0.7]).reset_index()[['year', '30%', '70%']]
    quantiles_proxy_beta.rename(columns={'30%':'B1_1t','70%':'B1_2t'},inplace=True)

    beta_mktcap_group = pd.merge(df,quantiles_proxy_beta, on=['year'])
    quantiles_proxy_mktcap = df.groupby(['year','X1_group'])['MktCap'].describe(percentiles=[0.25, 0.5, 0.75])
    quantiles_proxy_mktcap=quantiles_proxy_mktcap.reset_index()[['year','X1_group' ,'25%', '50%', '75%']]
    quantiles_proxy_mktcap.rename(columns = {'25%':'B2_1_kt','50%':'B2_2_kt','75%':'B2_3_kt'},inplace=True)

    table20 = pd.merge(quantiles_proxy_mktcap,quantiles_proxy_beta, on=['year'])
    table20.rename(columns={'X1_group':'k'})
    return table20
table20=generate_table20_2()
# table20=table20.rename(columns={'X1_group':'k'})
table20

,year,X1_group,B2_1_kt,B2_2_kt,B2_3_kt,B1_1t,B1_2t
0,1988,1,4.497000,12.238748,35.290500,0.179025,0.658815
1,1988,2,12.379359,39.544500,140.742937,0.179025,0.658815
2,1988,3,25.022281,125.705125,781.277250,0.179025,0.658815
3,1989,1,4.294688,12.117062,33.937391,0.172073,0.694472
4,1989,2,13.942172,45.811875,180.489688,0.172073,0.694472
...,...,...,...,...,...,...,...
70,2011,2,247.067100,897.123450,3204.374130,0.805027,1.380558
71,2011,3,213.818040,512.198740,1268.774055,0.805027,1.380558
72,2012,1,28.507760,64.449570,434.951450,0.746809,1.312337
73,2012,2,251.962640,788.745700,2771.084990,0.746809,1.312337


**TABLE 5.20 Bivariate Dependent-Sort Breakpoints** 

In [43]:
# 这个也用到了data all_data_and_groups 改正需要重新使用一份数据集

all_data_and_breakpoints = pd.merge(all_data_and_groups,table20,on=['year','X1_group'])
all_data_and_breakpoints


,PERMNO,year,beta,rt_1,MktCap,X1_group,X2_group,B2_1_kt,B2_2_kt,B2_3_kt,B1_1t,B1_2t
0,10001,1988,0.255128,60.422576,6.362250,2,1,12.381844,39.68250,141.534063,0.179025,0.658815
1,10003,1988,0.280791,-61.669471,41.686000,2,3,12.381844,39.68250,141.534063,0.179025,0.658815
2,10005,1988,0.480927,-41.703377,0.785250,2,1,12.381844,39.68250,141.534063,0.179025,0.658815
3,10010,1988,0.362772,11.074425,15.140250,2,2,12.381844,39.68250,141.534063,0.179025,0.658815
4,10016,1988,0.520995,-19.131575,400.414000,2,4,12.381844,39.68250,141.534063,0.179025,0.658815
...,...,...,...,...,...,...,...,...,...,...,...,...
130915,93384,2012,1.162822,NaN,623.217840,2,3,252.312480,785.98488,2754.139575,0.746809,1.312337
130916,93392,2012,0.911327,NaN,192.888900,2,2,252.312480,785.98488,2754.139575,0.746809,1.312337
130917,93401,2012,0.979015,NaN,327.819971,2,2,252.312480,785.98488,2754.139575,0.746809,1.312337
130918,93418,2012,0.758444,NaN,555.542320,2,3,252.312480,785.98488,2754.139575,0.746809,1.312337


### 5.3.2 portofolio formation

In [44]:
all_data_and_breakpoints = pd.merge(all_data_and_groups,table20,on=['year','X1_group'])

def portfolio_d_3x4():
    df = all_data_and_breakpoints.copy()
    
    df['X2_group'] = np.select([
        (df['MktCap'] < df['B2_1_kt']),
        (df['MktCap'] > df['B2_1_kt']) & (df['MktCap'] <= df['B2_2_kt']),
        (df['MktCap'] > df['B2_2_kt']) & (df['MktCap'] <= df['B2_3_kt']),
        (df['MktCap'] > df['B2_3_kt'])],
        ['1','2','3','4'])
    return df[['PERMNO','year','beta','rt_1','MktCap','X1_group','X2_group']]
all_data_and_groups = portfolio_d_3x4()

def generate_table20(group_table):
    n = group_table.groupby(['year', 'X1_group', 'X2_group'])['PERMNO'].count().reset_index().rename(
            columns={'PERMNO': 'n_firms'})
    df = pd.pivot_table(n, index=['year', 'X2_group'], columns='X1_group')['n_firms'].reset_index()
    df.rename(columns={'1':'beta1','2':'beta2','3':'beta3'},inplace=True)
    return df
table20 = generate_table20(all_data_and_groups)
table20

X1_group,year,X2_group,beta1,beta2,beta3
0,1988,1,431,578,431
1,1988,2,430,578,431
2,1988,3,430,577,431
3,1988,4,431,578,431
4,1989,1,416,553,422
...,...,...,...,...,...
95,2011,4,277,369,280
96,2012,1,267,356,270
97,2012,2,266,356,269
98,2012,3,266,355,269


**TABLE 5.21 Bivariate Dependent-Sort Number of Stocks per Portfolio** This table presents the number of stocks in each of the 12 portfolios formed by sorting dependently into three 훽 groups and then into four MktCap groups. 